In [10]:
import torch
import random
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
import os
from torchvision.transforms import ToTensor
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
from tqdm import trange
import matplotlib.pyplot as plt
if torch.backends.mps.is_available():
    print("MPS is available on this device.")
    # Set the PyTorch device to MPS
    device = torch.device("cpu")

MPS is available on this device.


In [11]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        self.chunked = pd.read_csv(annotations_file, chunksize=60000)
        self.img_labels = next(self.chunked)
        self.transform = transform
        
    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        label = torch.tensor(self.img_labels.iloc[idx, :1].values, device = device)
        image = torch.tensor(self.img_labels.iloc[idx, 1:].values, dtype=torch.float, device = device)
        return image, label


class Mnet(nn.Module):
    def __init__(self):
        super(Mnet, self).__init__()
        self.l1 = nn.Linear(784, 128)
        self.act = nn.ReLU()
        self.l2 = nn.Linear(128, 30)
        self.l3 = nn.Linear(30,10)
    def forward(self, x):
        x = self.l1(x)
        x = self.act(x)
        x = self.l2(x)
        x = self.act(x)
        x = self.l3(x)
        return x

def validate_epoch(model, dat):
    accs = [batch_accuracy(model(x),y) for x,y in dat]
    return round(torch.stack(accs).mean().item(), 4)

def batch_accuracy(x, y):
    preds = softmax(x)
    predicted_value = torch.argmax(preds, dim=1)
    trgts = y.flatten()
    bools = predicted_value == trgts
    accs = bools.to(torch.float).mean()
    return accs

In [12]:
dl = CustomImageDataset("data/mnist_train.csv")
valid_dl = CustomImageDataset("data/mnist_test.csv")
train_data = DataLoader(dl, batch_size=1, shuffle=True)
test_data = DataLoader(valid_dl, batch_size=128, shuffle=True)
#model = nn.Sequential(nn.Linear(784, 128), nn.ReLU(), nn.Linear(128, 10), nn.ReLU())
model = Mnet().to(device)
opt = optim.Adam(model.parameters()) #optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs = 1
losses, accuracies = [], [] 
for i in (r := trange(epochs)):
    for x, y in train_data:
        p = model(x)
        print(p)
        loss = loss_fn(p, y.flatten())
        losses.append(loss)
        loss.backward()
        opt.step()
        opt.zero_grad()
    accuracy = validate_epoch(model, test_data)
    accuracies.append(accuracy)
    r.set_description(f"accuracy: {accuracy}, loss: {loss.data}")

  0%|                                                                                                                                | 0/1 [00:00<?, ?it/s]

tensor([[ 8.9324,  1.4860, -0.4533, -2.4107,  1.1790, -0.9236, -3.5147,  2.6154,
         -0.3931, -6.8113]], grad_fn=<AddmmBackward0>)
tensor([[  3.0559,   5.7035,  -0.4640,   0.3753,   5.2493,   0.3847,  -9.6623,
           8.2119,  -2.5167, -10.5971]], grad_fn=<AddmmBackward0>)
tensor([[ -3.6868, -14.2845,  -9.2406,   0.7866,   4.0028,  -1.3254,   1.1765,
          17.9734,   3.4673,  -0.3478]], grad_fn=<AddmmBackward0>)
tensor([[ 0.7019, -7.8209, -2.8193, -3.1416,  1.7921, -2.7071, -1.4478,  7.5003,
         -0.8569, -5.4206]], grad_fn=<AddmmBackward0>)
tensor([[-12.2860,   2.6410, -17.3597,   3.3761,   3.2431,  -2.4238,   1.5572,
          10.9634,  -5.1915, -14.5719]], grad_fn=<AddmmBackward0>)
tensor([[ -2.2110,   5.0456, -11.8677,   3.1505,   0.0306,   1.8937,   0.5088,
           0.4580,  -9.7896, -11.6746]], grad_fn=<AddmmBackward0>)
tensor([[ -5.2011,   2.4168, -14.4231,  16.2205,   6.3922,  17.0053,  11.1501,
         -14.5378, -12.3238, -24.7085]], grad_fn=<AddmmBackward0>

In [123]:
plt.plot(losses)

torch.Size([128, 784])